# Importing required libraries

`sqlalchemy` is a "toolkit" that allows access to SQL databases through python. SQL tables can be queried and directly brought to python objects without the need of an intermediate export-import step (such as creating a csv from MySQL Workbench).

Converting data between incompatible type systems is called Object-relational mapping, so you might see `sqlalchemy` referred to as an ORM tool.

Here we'll see the basic workflow for connecting to a local database and transforming it to a `pandas` dataframe. Explore more features in the docs https://www.sqlalchemy.org/

MySQL has its own ORM tool for interacting with python, called `Connector/Python` (https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html). The advantage of `sqlalchemy` is that it can also work with other DBMS such as Oracle, Postgres, etc.

In [ ]:
#!pip3 install pymsql

#we need to install into our Conda DA_Env :

#conda install -c anaconda pymysql

#and

# conda install -c anaconda sqlalchemy

In [1]:
import pandas as pd
import getpass
import sqlalchemy as sa

# To connect and extract tables from SQL database.

In [2]:
driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'

Your password:········


In [3]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [4]:
db_connection = sa.create_engine(connection_string)

In [5]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

['apps', 'bank', 'information_schema', 'lab_db', 'lab_erd', 'lab_group_db', 'lab_st_p', 'mysql', 'performance_schema', 'sakila', 'sys', 'world']


retrieve data from MYSQL into Pandas Data Frame

In [6]:
bank_loans = pd.read_sql_query('SELECT * FROM bank.loan;', db_connection)
bank_loans.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


Passing Parameter into a query

In [14]:
loan_id_threshold=5300
query = 'SELECT * FROM bank.loan where loan_id<'+str(loan_id_threshold)
filtered_loans = pd.read_sql_query(query, db_connection)
filtered_loans.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,940105,80952,24,3373.0,A
1,5170,1071,940120,253200,60,4220.0,C
2,5285,1603,940206,78936,12,6578.0,A
3,5189,1166,940207,149040,48,3105.0,A
4,5130,813,940511,24312,12,2026.0,A


 Retreiving data into python object (without Pandas)

In [8]:
result = engine.execute('SELECT * FROM loan')
for row in result:
    print(row)

NameError: name 'engine' is not defined

# Create a database in Mysql and save a table from Pandas into that database.

#### Create Database and commit

In [15]:
db_connection.execute("create database if not exists lab_db_python_sql;")

In [16]:
db_connection.execute("commit")

#### Inspect the database

In [17]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

['apps', 'bank', 'information_schema', 'lab_db', 'lab_db_python_sql', 'lab_erd', 'lab_group_db', 'lab_st_p', 'mysql', 'performance_schema', 'sakila', 'sys', 'world']


#### Write Pandas Data Frame to Database

In [18]:
test_table = pd.DataFrame({"a" : [1,2,3], "b" : [4,5,6]})
test_table

,a,b
0,1,4
1,2,5
2,3,6


In [19]:
test_table.to_sql(name="test_table", con=db_connection, schema="lab_db_python_sql")

ValueError: Table 'test_table' already exists.

#### Calling Stored procedure from Sakilla Database

In [22]:
db_connection.execute('USE sakila;')

In [25]:
stock = db_connection.execute("CALL film_in_stock (1,1,@x);")
for row in stock:
    print(row)

OperationalError: (pymysql.err.OperationalError) (1046, 'No database selected')
[SQL: CALL film_in_stock (1,1,@x);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

#### Retreiving data from a View in a database

In [21]:
actor_info = pd.read_sql_query('SELECT * FROM sakila.actor_info;', db_connection)
actor_info.head()

,actor_id,first_name,last_name,film_info
0,1,PENELOPE,GUINESS,Animation: ANACONDA CONFESSIONS; Children: LAN...
1,2,NICK,WAHLBERG,Action: BULL SHAWSHANK; Animation: FIGHT JAWBR...
2,3,ED,CHASE,"Action: CADDYSHACK JEDI, FORREST SONS; Classic..."
3,4,JENNIFER,DAVIS,Action: BAREFOOT MANCHURIAN; Animation: ANACON...
4,5,JOHNNY,LOLLOBRIGIDA,"Action: AMADEUS HOLY, GRAIL FRANKENSTEIN, RING..."


In [27]:
view = db_connection.execute("SELECT 'actor_info','actor_id' FROM sakila.actor_info;")

In [28]:
for row in view:
    print(row)

('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
('actor_info', 'actor_id')
(